In [1]:
from sklearn.tree import DecisionTreeClassifier
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB


clfs = {
    "CART": DecisionTreeClassifier(random_state=1234),
    "EBM": ExplainableBoostingClassifier(),
    "LR": LogisticRegression(random_state=1234),
    "GNB": GaussianNB()
    
}

In [2]:
# datasets = ['breast', 'campus', 'churn', 'climate',
#             'compas', 'diabetes', 'german', 'heart',
#             'stroke', 'student', 'thoracic', 'water']
#fix compas dataset
#fix german dataset
#fix thoracic dataset

In [3]:
datasets = ['compas']

In [4]:
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold
n_datasets = len(datasets)
n_splits = 5
n_repeats = 2
rskf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=1234)

auc_scores = np.zeros((len(clfs), n_datasets, n_splits * n_repeats))

In [5]:
from sklearn.base import clone 
from sklearn import metrics
import pandas as pd

import helper
import importlib
importlib.reload(helper)
from sklearn.pipeline import make_pipeline

for data_id, dataset in enumerate(datasets):
    X=pd.read_csv(f"datasets/cleaned/{dataset}_X.csv")
    X = X.drop("Unnamed: 0", axis=1)
    y = pd.read_csv(f"datasets/cleaned/{dataset}_y.csv")
    y = y.drop("Unnamed: 0", axis=1)

    features_types_df = pd.read_csv(f"datasets/cleaned/datatypes/{dataset}.csv")

    feature_inidices = list(map(int, list(features_types_df)))
    features_names = list(features_types_df.T[0])
    features_types = list(map(int, list(features_types_df.T[1])))

    preprocess = helper.select_preprocessing_for_many_feat(feature_inidices, features_types, features_names)

    for fold_id, (train, test) in enumerate(rskf.split(X, y)):
        for clf_id, clf_name in enumerate(clfs):
            clf = clone(clfs[clf_name])

            if(clf_name == "GNB" and dataset = "compas"):
                clf.fit(X.iloc[train], y.iloc[train])
                y_preds = clf.predict(X.iloc[test])
                fpr, tpr, thresholds = metrics.roc_curve(y.iloc[test], y_preds)
                auc_scores[clf_id, data_id, fold_id] = metrics.auc(fpr, tpr)
            else:
                clf_pipeline = make_pipeline(
                    preprocess,
                    clf
                )
                
                clf_pipeline.fit(X.iloc[train], y.iloc[train])
                y_preds = clf_pipeline.predict(X.iloc[test])
                fpr, tpr, thresholds = metrics.roc_curve(y.iloc[test], y_preds)
                auc_scores[clf_id, data_id, fold_id] = metrics.auc(fpr, tpr)

/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/interpret/utils/all.py:317: RuntimeWarning: Sparse data not fully supported, will be densified for now, may cause OOM
  warnings.warn(warn_msg, RuntimeWarning)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/interpret/utils/all.py:317: RuntimeWarning: Sparse data not fully supported, will be densified for now, may cause OOM
  warnings.warn(warn_msg, RuntimeWarning)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as sh

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

In [ ]:
auc_scores

array([[[0.56179687, 0.56639706, 0.58629902, 0.54360511, 0.5780271 ,
         0.58015625, 0.56833333, 0.5810049 , 0.57743969, 0.5448925 ]],

       [[0.56257813, 0.6020098 , 0.62556373, 0.60277158, 0.60907151,
         0.59960938, 0.60203431, 0.59666667, 0.60265899, 0.59639333]],

       [[0.50390625, 0.49875   , 0.50196078, 0.5       , 0.5       ,
         0.50070313, 0.5       , 0.5       , 0.5       , 0.49874687]],

       [[0.54546875, 0.53355392, 0.57245098, 0.54867148, 0.555084  ,
         0.5615625 , 0.54588235, 0.53659314, 0.55744341, 0.55007147]]])